In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

2023-04-21 08:40:49.324689: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-21 08:40:49.364807: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-21 08:40:49.366145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 08:40:53.401042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_labels = pd.read_csv('train_data/_annotations.csv')

train_labels['class'] = train_labels['class'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
    train_labels,
    directory='train_data',
    x_col='filename',
    y_col='class',
    target_size=(771,771),
    batch_size=32,
    class_mode='categorical')

Found 146 validated image filenames belonging to 5 classes.


/home/u191397/.local/lib/python3.9/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 1328 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
base_model = tf.keras.applications.ResNet152(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)


model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20)

Epoch 1/20


2023-04-21 08:41:07.495276: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - 134s 32s/step - loss: 0.8560 - accuracy: 0.4737
Epoch 2/20
4/4 [==============================] - 129s 30s/step - loss: 0.7408 - accuracy: 0.8000
Epoch 3/20
4/4 [==============================] - 91s 20s/step - loss: 0.6518 - accuracy: 0.8000
Epoch 4/20
4/4 [==============================] - 88s 23s/step - loss: 0.5324 - accuracy: 0.8000
Epoch 5/20
4/4 [==============================] - 99s 23s/step - loss: 0.5651 - accuracy: 0.8000
Epoch 6/20
4/4 [==============================] - 90s 20s/step - loss: 0.5105 - accuracy: 0.8000
Epoch 7/20
4/4 [==============================] - 90s 20s/step - loss: 0.5343 - accuracy: 0.8000
Epoch 8/20
4/4 [==============================] - 90s 20s/step - loss: 0.5059 - accuracy: 0.8000
Epoch 9/20
4/4 [==============================] - 89s 20s/step - loss: 0.5152 - accuracy: 0.8000
Epoch 10/20
4/4 [==============================] - 90s 20s/step - loss: 0.5027 - accuracy: 0.8000
Epoch 11/20
4/4 [=====================

In [ ]:
t_test_df = pd.read_csv("test_data/_annotations.csv")

In [6]:
t_test_datagen = ImageDataGenerator(rescale=1./255)
t_test_generator = t_test_datagen.flow_from_dataframe(
    t_test_df,
    directory="test_data",
    x_col='filename',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False)


preds = model.predict_generator(t_test_generator, steps=None)

missing_filenames = set(t_test_generator.filenames) - set(t_test_df['filename'])
if missing_filenames:
    print(f'Removing {len(missing_filenames)} missing filenames from the test CSV file.')
    t_test_df = t_test_df[~t_test_df['filename'].isin(missing_filenames)]

/home/u191397/.local/lib/python3.9/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 141 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/home/u191397/tmp/ipykernel_3082191/3047856893.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(t_test_generator, steps=None)
2023-04-21 09:12:51.680800: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [7]:
li = []
for i in preds.ravel():
    if i>0.5:
        li.append(1)
    else:
        li.append(0)
     

df = pd.DataFrame({'filename': t_test_generator.filenames[:len(preds)], 'class': li})

test_inst_df=pd.merge(df, t_test_df, on='filename', how='inner')

In [8]:
model.save('models/resnet.h5')